Parsing of all medias in the form of {name, country, blues, factuality}

The second script (after that) adds a link to the name field

In [ ]:
from bs4 import BeautifulSoup
import requests
import json

def main(i):
    url = f"https://mediabiasfactcheck.com/filtered-search/?pg={i}"  # Замените на нужную ссылку

    response = requests.get(url)
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', class_="mbfc-table")
    rows = table.find_all('tr')

    for row in rows:
        cells = row.find_all('td')
        cell_values = [cell.get_text(strip=True) for cell in cells]
        if cell_values:
            print(cell_values[0], cell_values[1], cell_values[2], cell_values[3])
            data = {
                "name": cell_values[0],
                "country": cell_values[3],
                "bias": cell_values[1],
                "factuality": cell_values[2]
            }
            cell_values[0] = (cell_values[0]).replace("-", '%').replace('\\','!').replace(' ', '_').replace('/', '$')
            with open(f"labels/{cell_values[0]}.json", "w", encoding="utf-8") as file:
                json.dump(data, file, indent=4)

for i in range(50, 69):
    main(i)



Parsing of all media links, in the form of a pair (name, link)

The links file.json is simply copied from the archive to mediabiasfactcheck.com

In [ ]:
import json
import os

try:
    with open("links.json", "r", encoding="utf-8") as f:
        data = json.load(f)
except json.decoder.JSONDecodeError as e:
    error_position = e.pos
    error_character = e.doc[e.pos]
    print(f"Проблемный символ: '{error_character}' (позиция {error_position})")


with open("links_.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

Assembling a full-fledged json file with the link field

In [ ]:
import json
import os

count = 0
# connecting to an old dataset, removing repetitions
dir1 = "labels"
dir2 = "links"
for file1 in os.listdir(dir1):
    for file2 in os.listdir(dir2):

            if(file1 == file2):
                try:

                    count += 1
                    with open(os.path.join(dir1, file1), "r", encoding="utf-8") as f1:
                        data1 = json.load(f1)
                    with open(os.path.join(dir2, file2), "r", encoding="utf-8") as f2:
                        data2 = json.load(f2)
                    data = {
                        "name": data1["name"],
                        "country": data1["country"],
                        "bias": data1['bias'],
                        "factuality": data1['factuality'],
                        "link": data2['link']
                    }
                    name = file1
                    with open(f"no_repeat/{name}", "w", encoding="utf-8") as f:
                        json.dump(data, f, indent=4)
                except json.JSONDecodeError:
                    print(f"ERR WITH {file1} and {file2}")


print(count)


In [ ]:
# # parsing content from the first page of the media site (10-15 articles on average)
import os
import requests
from bs4 import BeautifulSoup
import json

dataset_dir = 'no_repeat'
#dataset_dir = 'test'
json_files = [f for f in os.listdir(dataset_dir) if f.endswith('.json')]

def get_data(url):
    try:
        response = requests.get(url, timeout=3)
        text = response.text
        content = BeautifulSoup(text, 'html.parser').get_text()
        return content
    except Exception as e:
        print(f"Bad link in {url}")
        with open("log.txt", 'a', encoding='utf-8') as f:
             f.write(f"Bad link in {url}")
small_control = []
for json_file in json_files:
    with open(os.path.join(dataset_dir, json_file), 'r', encoding='utf-8') as f:
        data = json.load(f)
        url = (data['link']).replace("https://", "")
        url = "https://" + url
        content = get_data(url)
        if content:
            data['content'] = content

            # (if the text is too small, we delete it)
            # this means that it is not possible to automate parsing
            # tc or the media site is blocked
            # or an agreement for data processing and cookies pops up
            # and the text of either the error or this agreement is parsed
            # such garbage parsing is not needed. Delete

            if len(content) < 150:
                print(f"small content in {json_file}")
                small_control.append(json_file)

        else:
            data['content'] = ""

    with open(os.path.join(dataset_dir, json_file), 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"Content added to {json_file}.")

print(small_control)
print(len(small_control))

Пост обработка.

Добавление numeric к bias и factuality

In [ ]:
import json
import os

directory = 'gold'
def add_num_once(filep):
    file = os.path.join(filep)
   
    with open(file, 'r', encoding="utf-8") as f:
        data = json.load(f)
# Adding numeric fields to existing text fields
    try:
        bias_text = data['bias']

        if bias_text == 'Left':
            data['bias (numeric)'] = 0
        elif bias_text == 'Left-Center':
            data['bias (numeric)'] = 1
        elif bias_text == 'Center' or bias_text == 'Least Biased':
            data['bias (numeric)'] = 2
        elif bias_text == 'Right-Center':
            data['bias (numeric)'] = 3
        elif bias_text == 'Right':
            data['bias (numeric)'] = 4
        else :
            data['bias (numeric)'] = -1
    except IndexError:
        print(file)

    try:
        factuality_text = data['factuality']
        if factuality_text == 'Very High':
            data['factuality (numeric)'] = 0
        elif factuality_text == 'High':
            data['factuality (numeric)'] = 1
        elif factuality_text == 'Mostly Factual':
            data['factuality (numeric)'] = 2
        elif factuality_text == 'Mixed':
            data['factuality (numeric)'] = 3
        elif factuality_text == 'Low':
            data['factuality (numeric)'] = 4
        elif factuality_text == 'Very Low':
            data['factuality (numeric)'] = 5
        else:
            data['factuality (numeric)'] = -1
    except IndexError:
        print(file)
   # data[0]['Link'] = 'https://' + data[0]['Link']

    with open(file, 'w') as f:
        json.dump(data, f, indent=4)


for filename in os.listdir(directory):
  
    filepath = os.path.join(directory, filename)
   
    if filepath.endswith('.json'):
        add_num_once(filepath)

Randomization of the name and id field

In [ ]:
import os
import random
import string
import json

folder_path = 'dir'


for filename in os.listdir(folder_path):
    old_file_name = filename


    new_file_name = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8)) + '.json'
    new_file_path = os.path.join(folder_path, new_file_name)


    os.rename(folder_path + '/' + old_file_name, new_file_path)

    with open(new_file_path, 'r+', encoding='utf-8') as f:
        data = json.load(f)
        data['id'] = new_file_name[:-5]
        f.seek(0)
        json.dump(data, f, indent=2)
        f.truncate()

Bringing the file to the desired form.

Setting up the order of fields for convenient parsing quality control

In [ ]:
import os
import json
import shutil
import random
import string

path = 'gold'

for filename in os.listdir(path):

    if filename.endswith(".json"):

        with open(os.path.join(path, filename), "r", encoding='utf-8') as f:
            content = json.load(f)
        data = {
            "id": ''.join(random.choices(string.ascii_letters + string.digits, k=8)),
            "name": content["name"],
            "country": content["country"],
            "bias": content["bias"],
            "bias (numeric)": content["bias (numeric)"],
            "factuality": content["factuality"],
            "factuality (numeric)": content["factuality (numeric)"],
            "link": content["link"],
            "content": content["content"]
        }
        with open(os.path.join(path, filename), "w", encoding='utf-8') as f:
            json.dump(data, f, indent=4)

Later, a new parsing strategy was chosen. 

It consists in collecting all links from the main media site, removing among them those that lead to another domain and those that are less than 50 characters long (these are links from the menu and junk links to advertising). It is assumed that after such a selection, links to articles remain. 

Next, click on them and parse the text from the articles page.